<a href="https://colab.research.google.com/github/vaibhavsundharam/Distracted-Driver-Detection-using-Deep-Learning/blob/master/Ensamble_AML_project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import sys
import os
from __future__ import print_function, division
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms as transforms
import torchvision.models as models
from skimage.transform import resize
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Set device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_project" 
customized_path_to_your_project = 'AML/Project/'
sys_path = os.path.join(prefix, customized_path_to_your_project)
sys.path.append(sys_path)
print(f"System path: {sys_path}")

System path: /content/gdrive/My Drive/AML/Project/


In [ ]:
# reading the CSV file with driver-class mapping for training
root_dir = sys_path+"Coding"
driver_imgs_csv = sys_path+"Coding/CSV/driver_imgs_list.csv"

print("Root directory Location: {}".format(root_dir)+"\n")
print("CSV file Location: {}".format(driver_imgs_csv)+"\n")


drivers_imgs_file = pd.read_csv(driver_imgs_csv,engine='python')
print(drivers_imgs_file)

n=0
img_name=drivers_imgs_file.iloc[n,2]
img_class=drivers_imgs_file.iloc[n,1]
print("Image name: {}".format(img_name))
print("Image class: {}".format(img_class[1]))


Root directory Location: /content/gdrive/My Drive/AML/Project/Coding

CSV file Location: /content/gdrive/My Drive/AML/Project/Coding/CSV/driver_imgs_list.csv

      subject classname            img
0        p002        c0  img_44733.jpg
1        p002        c0  img_72999.jpg
2        p002        c0  img_25094.jpg
3        p002        c0  img_69092.jpg
4        p002        c0  img_92629.jpg
...       ...       ...            ...
22419    p081        c9  img_56936.jpg
22420    p081        c9  img_46218.jpg
22421    p081        c9  img_25946.jpg
22422    p081        c9  img_67850.jpg
22423    p081        c9   img_9684.jpg

[22424 rows x 3 columns]
Image name: img_44733.jpg
Image class: 0


# Data loading

In [ ]:
class DistractedDriverDataset(Dataset): # custom class to explore the dataset 

  def __init__(self, csv_file_loc, root_dir, transform=None):
    self.driver_imgs_file = pd.read_csv(csv_file_loc) #  reading the CSV file
    self.root_dir = root_dir # root directory of the images 
    self.transform = transform # transformations if any

  def __len__(self):  # returns the length of the entire training set
    return len(self.driver_imgs_file)

  def __getitem__(self, idx):    
    img_name=os.path.join(self.root_dir,self.driver_imgs_file.iloc[idx,1]) # featching the file name     
    img_tensor=self.transform(Image.open(img_name)) # reading the image
    
     
    
    return img_tensor, img_name

In [ ]:
'''class DistractedDriverDataset(Dataset): # custom class to explore the dataset 

  def __init__(self, csv_file_loc, root_dir, transform=None):
    self.driver_imgs_file = pd.read_csv(csv_file_loc) #  reading the CSV file
    self.root_dir = root_dir # root directory of the images 
    self.transform = transform # transformations if any

  def __len__(self):  # returns the length of the entire training set
    return len(self.driver_imgs_file)

  def __getitem__(self, idx):    
    img_name=os.path.join(self.root_dir,self.driver_imgs_file.iloc[idx,1],self.driver_imgs_file.iloc[idx,2]) # featching the file name    
     
    driver_name=self.driver_imgs_file.iloc[idx,0] # driver name
    #print(img_name, driver_name)
    img_tensor=self.transform(Image.open(img_name)) # reading the image
    
    img_class=int(self.driver_imgs_file.iloc[idx,1][1])# fetching the image class 
    
    return img_tensor, img_class  #img_name[64:77]   #,driver_name


In [ ]:
# Here we create two seperate instances of the Distracted driver data set. One will hold the data set for training and the other will hold data for validation

print("Root directory:",root_dir,"\n")

test_dataset=DistractedDriverDataset(csv_file_loc="/content/Test_csv.csv",root_dir=root_dir+"/Images/Test",
                                transform=transforms.Compose([transforms.Resize((244,244)),transforms.ToTensor()]))
len_test_dataset=len(test_dataset)
print("Total number of elements in training data:", len_test_dataset)




Root directory: /content/gdrive/My Drive/AML/Project/Coding 

Total number of elements in training data: 69


In [ ]:
# loading training and validation data for the neural network
test_loader=DataLoader(test_dataset, batch_size=32,shuffle=True, num_workers=16) # x.shape--->torch.Size([32, 3, 244, 244])

'''for x, y in test_loader:
  print(x.shape)
  print(y[0][60:])'''
  
  


'for x, y in test_loader:\n  print(x.shape)\n  print(y[0][60:])'

#Helper functions

In [ ]:
# solver function for NN training
def Solver_NN (model, name, train_loader, optim, criterion, len_train_dataset, len_val_dataset, epoch=51, lr=1e-1):
  print("Solver Initiated")
  model=model.to(device) # sending model to GPU
  print("Model successfully sent to the GPU\n")  

  train_acc_list=list()
  val_acc_list=list()
  epoch_list=list()
  train_loss_list=list()
  val_loss_list=list()
  total_step = len(train_loader)
  best_epoch=None
  #curr_lr = learning_rate
  counter=0
  for e in range(epoch):
    model.train()
    loss_epoch=0     
    epoch_list.append(e)   

    for i,(x,y) in enumerate(train_loader):      
      x = x.to(device)
      y = y.to(device)    
      #print("x is:",x.shape)     
      #print("y is:",y.shape)

      

      #forward pass
      y_pred=model(x)
      #print("y_pred is:",y_pred.shape)
      loss=criterion(y_pred,y)
      #print("loss:",loss)

      #bacward pass
      optim.zero_grad()
      #loss.to(device)
      loss.backward()
      optim.step()
      
      loss_epoch += float(loss.item())
      #print("loss epoch:",loss_epoch)    
         
      #print(f"Training loss in epoch {e} is {(loss_epoch/len_train_dataset)*1000}") 
      #train_loss_list.append((loss_epoch/len_train_dataset)*1000)

      if (i+1) % 100 == 0:
        print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(counter+1, epoch, i+1, total_step, loss.item()))
    counter+=1
      
    print(f"Training loss in epoch {e} is {(loss_epoch/len_train_dataset)*1000}") 
    train_loss_list.append((loss_epoch/len_train_dataset)*1000)
    model.eval()
    with torch.no_grad():      
      val_acc, val_loss = get_model_acc_and_loss(True,model,criterion,val_loader)
      print(f"Validation loss in epoch {e} is {(val_loss/len_val_dataset)*1000}")
      val_loss_list.append((val_loss/len_val_dataset)*1000)
      train_acc = get_model_acc_and_loss(False,model,criterion,train_loader)
      train_acc_list.append(train_acc)
      val_acc_list.append(val_acc)
      print(f'Validation accuracy: {val_acc}, Train accuracy: {train_acc}') 

      if best_epoch==None:
        best_epoch=e        
        best_val=val_acc
        torch.save(model.state_dict(), root_dir+'/Saved Weights/'+name+" epoch "+str(e))
        print("File saved successfully\n")
      elif val_acc>best_val:
        best_epoch=e        
        best_val=val_acc
        torch.save(model.state_dict(), root_dir+'/Saved Weights/'+name+" epoch "+str(e))
        print("File saved successfully\n")
      else:
        print("\n")

  print(f'Best epoch:{best_epoch}, Best Validtion accuracy:{best_val}')

  fig, axs = plt.subplots(2)  
  fig.tight_layout(pad=3.0)
  axs[0].plot(epoch_list,train_acc_list, 'r--' ,epoch_list,val_acc_list,'g')
  axs[0].set_title('Accuracy vs. epochs')  
  axs[1].plot(epoch_list,train_loss_list, 'r--' ,epoch_list,val_loss_list,'g')
  axs[1].set_title('Loss vs. epochs')
  

  plt.savefig(name+".png")

  
  
  return model 

#Extra layer

In [ ]:
class extra_layer(nn.Module):
    
    def __init__(self):
        super().__init__()               
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, 512),
            nn.Linear(512, 10))
    def forward(self, x):        
        return self.fc(x)

##Ensamble

In [ ]:
class Ensemble(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD, modelE, nb_classes=10):
        super(Ensemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.modelD = modelD
        self.modelE = modelE  
    
        
    def forward(self, x):
        A = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        

        B = self.modelB(x.clone())
               

        C = self.modelC(x.clone())
        

        D = self.modelD(x.clone())
        

        E = self.modelE(x)    

        return A, B, C ,D ,E


In [ ]:

modelA = nn.Sequential(torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=False), extra_layer())
modelA.load_state_dict(torch.load("/content/gdrive/My Drive/AML/Project/Coding/With weights/Weights/VGG16 epoch 1"))
#print(modelA)

modelB = nn.Sequential(torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False), extra_layer())
modelB.load_state_dict(torch.load("/content/gdrive/My Drive/AML/Project/Coding/With weights/Weights/RESNET18 epoch 8"))
#print(modelB)

modelC = nn.Sequential(torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=False), extra_layer())
modelC.load_state_dict(torch.load("/content/gdrive/My Drive/AML/Project/Coding/With weights/Weights/RESNET34 epoch 8 with weights"))
#print(modelC)

modelD = nn.Sequential(torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False), extra_layer())
modelD.load_state_dict(torch.load("/content/gdrive/My Drive/AML/Project/Coding/With weights/Weights/ALEXNET epoch 9"))
#print(modelD)

modelE = nn.Sequential(torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=False), extra_layer())
modelE.load_state_dict(torch.load("/content/gdrive/My Drive/AML/Project/Coding/With weights/Weights/MOBILENET_V2 epoch 6"))
#print(modelE)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


<All keys matched successfully>

In [ ]:
# Instantiate ensamble
model = Ensemble(modelA, modelB, modelC, modelD, modelE, nb_classes=10)

#criterion and optimizer

lr=1e-2
criterion=nn.CrossEntropyLoss(reduction='mean') # loss criterion 
optim=torch.optim.SGD(model.parameters(), lr=lr) # optimizer 

In [ ]:
'''def ensemble_model_acc(model, loader):      
  ys=[]
  y_predsA=[]
  y_predsB=[]
  y_predsC=[]
  y_predsD=[]
  y_predsE=[]  
  model.to(device)  
  for x, y in loader:    
    x=x.to(device)
    y=y.to(device)
    ys.append(y)

    y_predA, y_predB, y_predC, y_predD, y_predE=model(x)
    
    y_predsA.append(torch.argmax(y_predA, dim=1))
    y_predsB.append(torch.argmax(y_predB, dim=1))
    y_predsC.append(torch.argmax(y_predC, dim=1))
    y_predsD.append(torch.argmax(y_predD, dim=1))
    y_predsE.append(torch.argmax(y_predE, dim=1))
  
  
  y_predA=torch.cat(y_predsA, dim=0)
  y_predB=torch.cat(y_predsB, dim=0)
  y_predC=torch.cat(y_predsC, dim=0)
  y_predD=torch.cat(y_predsD, dim=0)
  y_predE=torch.cat(y_predsE, dim=0)
  
  Y=torch.stack((y_predA, y_predB, y_predC, y_predC, y_predD, y_predE,y_predE ), dim=1)
  y=torch.cat(ys, dim=0)

  Y=Y.cpu()
  y=y.cpu()
  
  Y=np.asarray(Y)
  print(Y)
  y=np.asarray(y)
  print(y)
  
  combine=list() 

  for i in range(Y.shape[0]):
      x=np.bincount(Y[i]).argmax()
      combine.append(x)

  
  combine=np.asarray(combine)
  print(combine)

  ans=(y==combine).sum()/len(y)
  print(ans)

  return True

In [ ]:
def ensemble_model_acc(model, loader):      
  ys=[]
  y_predsA=[]
  y_predsB=[]
  y_predsC=[]
  y_predsD=[]
  y_predsE=[] 
  img_names=[] 
  model.to(device)  

  for x, y in loader:    
    x=x.to(device)    
    ys.append(y)    

    y_predA, y_predB, y_predC, y_predD, y_predE=model(x)
    
    y_predsA.append(torch.argmax(y_predA, dim=1))
    y_predsB.append(torch.argmax(y_predB, dim=1))
    y_predsC.append(torch.argmax(y_predC, dim=1))
    y_predsD.append(torch.argmax(y_predD, dim=1))
    y_predsE.append(torch.argmax(y_predE, dim=1))
  
  
  y_predA=torch.cat(y_predsA, dim=0)
  y_predB=torch.cat(y_predsB, dim=0)
  y_predC=torch.cat(y_predsC, dim=0)
  y_predD=torch.cat(y_predsD, dim=0)
  y_predE=torch.cat(y_predsE, dim=0)
  
  Y=torch.stack((y_predA, y_predB, y_predC, y_predC, y_predD, y_predE ), dim=1)
  Y=Y.cpu()  
  Y=np.asarray(Y)
  print(Y)
  
  for i in range (len(ys)):
    for loc in range(len(ys[i])):      
      img_name=ys[i][loc]
      img_names.append(img_name[60:])
    
  #print(f"Image names: {img_names}")

  combine=list()
  for i in range(Y.shape[0]):
      x=np.bincount(Y[i]).argmax()
      combine.append(x)

  
  combine=np.asarray(combine)
  #print(combine)
  
  result=zip(img_names,combine)
  print(list(result))
  #ans=(y==combine).sum()/len(y)
  #print(ans)

  return True

In [ ]:
model.eval()
with torch.no_grad():
  test_acc = ensemble_model_acc(model, test_loader)
